# Playground to experiment

In [1]:
import gensim
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import kmeans
from pre_process import extract_mapped_doc2vecs, Doc2vec_wrapper

/home/prathamesh/myvenv/_tml_clustering_/lib/python3.5/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


## Compute optimal minPts and eps value for DBSCAN

Using the heuristic suggested in Section 4.2 of the [DBSCAN paper](https://www.aaai.org/Papers/KDD/1996/KDD96-037.pdf).

In [2]:
from utils import compute_optimal_k_dists
def get_kdist_plot(data, k):
    kdists = sorted(compute_optimal_k_dists(data, k), reverse=True)
    #len(kdists)

    plt.plot(range(1, len(kdists) + 1), kdists, '-ro')
    return plt.show

### ETD docvectors

In [ ]:
DOCVEC_PATH = '../obj/etd/doc2vec/abstracts_etd_doc2vec_all_docs30961_docs'

docvec_model = gensim.models.doc2vec.Doc2Vec.load(DOCVEC_PATH)

doc_vectors, keys = extract_mapped_doc2vecs(docvec_model)

get_kdist_plot(np.array(doc_vectors), 3)

23140


In [ ]:
get_kdist_plot(np.array(doc_vectors), 4)

In [ ]:
get_kdist_plot(np.array(doc_vectors), 5)

In [ ]:
get_kdist_plot(np.array(doc_vectors), 8)

In [ ]:
get_kdist_plot(np.array(doc_vectors), 10)